In [ ]:
!pip install -r /content/drive/MyDrive/requirements.txt

In [ ]:
!pip install pypdf

In [ ]:
!pip install sentence-transformers

In [ ]:
#Pinecone and hugging face API keys
%env PINECONE_API_KEY=
%env HUGGINGFACE_API_KEY=

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
import os
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.document_loaders import PyPDFLoader
class ChatBot():

  loader = PyPDFLoader('/content/drive/MyDrive/INPT _infos.pdf')
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
  docs = text_splitter.split_documents(documents)

  embeddings = HuggingFaceEmbeddings()


  # Initialize Pinecone client
  pinecone.init(api_key= os.getenv('PINECONE_API_KEY'))

  # Define Index Name
  index_name = "chatbot"

  # Checking Index
  if index_name not in pinecone.list_indexes():
    docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

  else:
    docsearch = Pinecone.from_existing_index(index_name, embeddings)


  repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
  llm = HuggingFaceHub(
      repo_id=repo_id,
      model_kwargs={"temperature": 0.8, "top_k": 50},
      huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')

  )
  template = """
  You are an assistant specializing in the Institut National des Postes et Télécommunications (INPT).
  Your role is to provide information and answer questions  related only to INPT, including its programs, research, and organizational details.
  If a question is outside your area of expertise, politely inform the user that you can only assist with questions related to INPT.

  Context: {context}
  Question: {question}
  Answer:

  """

  prompt = PromptTemplate(
      template=template,
      input_variables=["context", "question"]
  )

  rag_chain = (
    {"context": docsearch.as_retriever(),  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
  )

In [ ]:
bot = ChatBot()
user_input = input("Ask me anything: ")
result = bot.rag_chain.invoke(user_input)
print(str(result))